https://towardsdatascience.com/beyond-a-b-testing-multi-armed-bandit-experiments-1493f709f804
https://www.kaggle.com/cstorm3000/multi-armed-bandits-from-scratch

In [1]:
import numpy as np
import pandas as pd

#widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

#plots
import matplotlib.pyplot as plt
from plotnine import *
from mizani import *

#stats
import scipy as sp
import statsmodels as sm

import warnings
warnings.filterwarnings('ignore')
import collections

In [2]:
def gen_campaigns(p1,p2,nb_days,scaler,seed):
    #generate fake data
    np.random.seed(seed)
    ns = np.random.triangular(50,100,150,size=nb_days*2).astype(int)
    np.random.seed(seed)
    es = np.random.randn(nb_days*2) / scaler

    n1 = ns[:nb_days]
    c1 = ((p1 + es[:nb_days]) * n1).astype(int)
    n2 = ns[nb_days:]
    c2 = ((p2 + es[nb_days:]) * n2).astype(int)
    conv_days = pd.DataFrame({'click_day':range(nb_days),'click_a':n1,'conv_a':c1,'click_b':n2,'conv_b':c2})

    conv_days =  conv_days[['click_day','click_a','click_b','conv_a','conv_b']]
    conv_days['cumu_click_a'] = conv_days.click_a.cumsum()
    conv_days['cumu_click_b'] = conv_days.click_b.cumsum()
    conv_days['cumu_conv_a'] = conv_days.conv_a.cumsum()
    conv_days['cumu_conv_b'] = conv_days.conv_b.cumsum()
    conv_days['cumu_rate_a'] = conv_days.cumu_conv_a / conv_days.cumu_click_a
    conv_days['cumu_rate_b'] = conv_days.cumu_conv_b / conv_days.cumu_click_b
    return conv_days

conv_days = gen_campaigns(p1 = 0.10,
                          p2 = 0.105,
                          nb_days = 24,
                          scaler=300,
                          seed = 1412) #god-mode 
conv_days.tail()

,click_day,click_a,click_b,conv_a,conv_b,cumu_click_a,cumu_click_b,cumu_conv_a,cumu_conv_b,cumu_rate_a,cumu_rate_b
19,19,80,129,8,14,2044,1866,192,188,0.093933,0.100750
20,20,100,101,9,10,2144,1967,201,198,0.093750,0.100661
21,21,135,77,13,7,2279,2044,214,205,0.093901,0.100294
22,22,112,93,11,10,2391,2137,225,215,0.094103,0.100608
23,23,97,72,9,7,2488,2209,234,222,0.094051,0.100498


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
class Arm(object):
    """
    Each arm's true click through rate is 
    modeled by a beta distribution.
    """
    def __init__(self, idx, a=1, b=1):
        """
        Init with uniform prior.
        """
        self.idx = idx
        self.a = a
        self.b = b
        
    def record_success(self):
        self.a += 1
        
    def record_failure(self):
        self.b += 1
        
    def draw_ctr(self):
        return np.random.beta(self.a, self.b, 1)[0]
    
    def mean(self):
        return self.a / (self.a + self.b)

In [9]:
def monte_carlo_simulation(arms, draw=100):
    """
    Monte Carlo simulation of thetas. Each arm's click through
    rate follows a beta distribution.
    
    Parameters
    ----------
    arms list[Arm]: list of Arm objects.
    draw int: number of draws in Monte Carlo simulation.
    
    Returns
    -------
    mc np.matrix: Monte Carlo matrix of dimension (draw, n_arms).
    p_winner list[float]: probability of each arm being the winner.
    """
    # Monte Carlo sampling
    alphas = [arm.a for arm in arms]
    betas = [arm.b for arm in arms]
    mc = np.matrix(np.random.beta(alphas, betas, size=[draw, len(arms)]))
    
    # count frequency of each arm being winner 
    counts = [0 for _ in arms]
    winner_idxs = np.asarray(mc.argmax(axis=1)).reshape(draw,)
    for idx in winner_idxs:
        counts[idx] += 1
    
    # divide by draw to approximate probability distribution
    p_winner = [count / draw for count in counts]
    return mc, p_winner

In [11]:
def thompson_sampling(arms):
    """
    Stochastic sampling: take one draw for each arm
    divert traffic to best draw.
    
    @param arms list[Arm]: list of Arm objects
    @return idx int: index of winning arm from sample
    """
    sample_p = [arm.draw_ctr() for arm in arms]
    idx = np.argmax(sample_p)
    return idx

In [12]:
def should_terminate(p_winner, est_ctrs, mc, alpha=0.05):
    """
    Decide whether experiument should terminate. When value remaining in
    experiment is less than 1% of the winning arm's click through rate.
    
    Parameters
    ----------
    p_winner list[float]: probability of each arm being the winner.
    est_ctrs list[float]: estimated click through rates.
    mc np.matrix: Monte Carlo matrix of dimension (draw, n_arms).
    alpha: controlling for type I error
    
    @returns bool: True if experiment should terminate.
    """
    winner_idx = np.argmax(p_winner)
    values_remaining = (mc.max(axis=1) - mc[:, winner_idx]) / mc[:, winner_idx]
    pctile = np.percentile(values_remaining, q=100 * (1 - alpha))
    return pctile < 0.01 * est_ctrs[winner_idx]

In [13]:
def k_arm_bandit(ctrs, alpha=0.05, burn_in=1000, max_iter=100000, draw=100, silent=False):
    """
    Perform stochastic k-arm bandit test. Experiment is terminated when
    value remained in experiment drops below certain threshold.
    
    Parameters
    ----------
    ctrs list[float]: true click through rates for each arms.
    alpha float: terminate experiment when the (1 - alpha)th percentile
        of the remaining value is less than 1% of the winner's click through rate.
    burn_in int: minimum number of iterations.
    max_iter int: maxinum number of iterations.
    draw int: number of rows in Monte Carlo simulation.
    silent bool: print status at the end of experiment.
    
    Returns
    -------
    idx int: winner's index.
    est_ctrs list[float]: estimated click through rates.
    history_p list[list[float]]: storing est_ctrs and p_winner.
    traffic list[int]: number of traffic in each arm.
    """
    n_arms = len(ctrs)
    arms = [Arm(idx=i) for i in range(n_arms)]
    history_p = [[] for _ in range(n_arms)]
    
    for i in range(max_iter):
        idx = thompson_sampling(arms)
        arm, ctr = arms[idx], ctrs[idx]

        # update arm's beta parameters
        if np.random.rand() < ctr:
            arm.record_success()
        else:
            arm.record_failure()

        # record current estimates of each arm being winner
        mc, p_winner = monte_carlo_simulation(arms, draw)
        for j, p in enumerate(p_winner):
            history_p[j].append(p)
            
        # record current estimates of each arm's ctr
        est_ctrs = [arm.mean() for arm in arms]
        
        # terminate when value remaining is negligible
        if i >= burn_in and should_terminate(p_winner, est_ctrs, mc, alpha):
            if not silent: print("Terminated at iteration %i"%(i + 1))
            break

    traffic = [arm.a + arm.b - 2 for arm in arms]
    return idx, est_ctrs, history_p, traffic

In [14]:
seed = 11
np.random.seed(seed)

ctrs = [0.04, 0.048, 0.03, 0.037, 0.044]
true_winner_idx = np.argmax(ctrs)
print("true_winner_idx:", true_winner_idx, ctrs)

(winner_idx, est_ctrs, history_p, traffic) = k_arm_bandit(ctrs, alpha=0.05, burn_in=1400)

true_winner_idx: 1 [0.04, 0.048, 0.03, 0.037, 0.044]
Terminated at iteration 14385


In [15]:
plot_history(ctrs, est_ctrs, history_p, 
             title="K-armed Bandit Algorithm (terminated in %i iterations)"%sum(traffic), rolling=100)

NameError: name 'plot_history' is not defined

In [17]:
import numpy as np
import pandas as pd

#widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

#plots
import matplotlib.pyplot as plt
from plotnine import *

#stats
import scipy as sp
import statsmodels as sm

In [22]:
class Arm:
    def __init__(self, true_p):
        self.true_p = true_p
        self.reset()
    def reset(self):
        self.impressions = 0
        self.actions = 0
    def get_state(self):
        return self.impressions,self.actions
    def get_rate(self):
        return self.actions / self.impressions if self.impressions >0 else 0.
    def pull(self):
        self.impressions+=1
        res = 1 if np.random.random() < self.true_p else 0
        self.actions+=res
        return res

In [23]:
a = Arm(0.1)
for i in range(100): a.pull()
a.get_state()

(100, 10)

In [24]:
class MusketeerEnv:
    def __init__(self, true_ps, avg_impressions):
        self.true_ps = true_ps
        self.avg_impressions = avg_impressions
        self.nb_arms = len(true_ps)
        self.vr_agent = BanditAgent()
        self.reset()
    def reset(self):
        self.t = -1
        self.ds=[]
        self.arms = [Arm(p) for p in self.true_ps]
        return self.get_state()
    def get_state(self):
        return [self.arms[i].get_state() for i in range(self.nb_arms)]
    def get_rates(self):
        return [self.arms[i].get_rate() for i in range(self.nb_arms)]
    def get_impressions(self):
        return int(np.random.triangular(self.avg_impressions/2,
                                    self.avg_impressions,
                                    self.avg_impressions*1.5))
    def step(self, ps):
        self.t+=1
        impressions = self.get_impressions()
        for i in np.random.choice(a=self.nb_arms,size=impressions,p=ps):
            self.arms[i].pull()
        self.record()
        return self.get_state()
    #use agent to calculate value remaining
    def value_remaining(self,n=1000,q=95):
        state = self.get_state()
        best_idx = np.argmax(self.vr_agent.thompson_stochastic(state))
        l=[]
        for i in range(n): l.append(self.vr_agent.thompson_one(state)[None,:])
        l = np.concatenate(l,0)
        l_max = l.max(1)
        l_best = l[:,best_idx]
        vs = (l_max - l_best)/l_best
        return np.percentile(vs,q)
    def record(self):
        d = {'t':self.t,'max_rate':0,'opt_impressions':0}
        for i in range(self.nb_arms):
            d[f'impressions_{i}'],d[f'actions_{i}'] = self.arms[i].get_state()
            d[f'rate_{i}'] = self.arms[i].get_rate()
            if d[f'rate_{i}'] > d['max_rate']: 
                d['max_rate'] = d[f'rate_{i}']
                d['opt_impressions'] = d[f'impressions_{i}']
        d['total_impressions'] = sum([self.arms[i].impressions for i in range(self.nb_arms)])
        d['opt_impressions_rate'] = d['opt_impressions'] / d['total_impressions']
        d['total_actions'] = sum([self.arms[i].actions for i in range(self.nb_arms)])
        d['total_rate'] = d['total_actions'] / d['total_impressions']
        d['regret_rate'] = d['max_rate'] - d['total_rate']
        d['regret'] = d['regret_rate'] * d['total_impressions']
        d['value_remaining'] = self.value_remaining()
        self.ds.append(d)
    def show_df(self):
        df = pd.DataFrame(self.ds)
        cols = ['t'] + [f'rate_{i}' for i in range(self.nb_arms)]+ \
               [f'impressions_{i}' for i in range(self.nb_arms)]+ \
               [f'actions_{i}' for i in range(self.nb_arms)]+ \
               ['total_impressions','total_actions','total_rate']+ \
               ['opt_impressions','opt_impressions_rate']+ \
               ['regret_rate','regret','value_remaining']
        df = df[cols]
        return df

In [25]:
class BanditAgent:
    def __init__(self):
        pass
    #thompsons
    def thompson_one(self,state):
        res = [np.random.beta(i[1]+1,i[0]-i[1]+1) for i in state]
        res = np.array(res)
        return res
    def thompson_stochastic(self,state,n=1000):
        l = []
        for i in range(n): l.append(self.thompson_one(state)[None,:])
        l = np.concatenate(l,0)
        is_max = l.max(1)[:,None] == l
        return is_max.mean(0)

In [26]:
env = MusketeerEnv(true_ps = [0.1,0.12,0.13], avg_impressions=400)
for i in range(1000):
    env.step([0.6,0.2,0.2])
env.get_rates()

[0.10044147304557087, 0.12003000750187547, 0.12968927021143883]

In [27]:
env.show_df().head()

,t,rate_0,rate_1,rate_2,impressions_0,impressions_1,impressions_2,actions_0,actions_1,actions_2,total_impressions,total_actions,total_rate,opt_impressions,opt_impressions_rate,regret_rate,regret,value_remaining
0,0,0.089844,0.127907,0.149254,256,86,67,23,11,10,409,44,0.107579,67,0.163814,0.041674,17.044776,0.588573
1,1,0.096154,0.130435,0.136986,468,161,146,45,21,20,775,86,0.110968,146,0.188387,0.026019,20.164384,0.517409
2,2,0.088456,0.111607,0.140187,667,224,214,59,25,30,1105,114,0.103167,214,0.193665,0.037019,40.906542,0.212233
3,3,0.089087,0.117647,0.128378,898,289,296,80,34,38,1483,152,0.102495,296,0.199595,0.025883,38.385135,0.290442
4,4,0.088768,0.107955,0.133880,1104,352,366,98,38,49,1822,185,0.101537,366,0.200878,0.032343,58.928962,0.111080


In [28]:
class BanditAgent:
    def __init__(self):
        pass
    #baselines
    def equal_weights(self,state):
        res = np.array([1/len(state) for i in range(len(state))])
        return res
    def randomize(self,state):
        res = np.random.rand(len(state))
        res /= res.sum()
        return res
    
    #stochastic policies
    def eps_greedy(self, state, t, start_eps=0.3, end_eps=0.01, gamma=0.99):
        eps = max(end_eps,start_eps * gamma**t)
        res = np.array([eps/len(state) for i in range(len(state))])
        best_idx = np.argmax([i[1]/i[0] for i in state]) if t > 0 else np.random.choice(range(len(state)))
        res[best_idx] += 1-eps
        return res
    def softmax(self, state, t, start_tau=1e-1, end_tau=1e-4, gamma=0.9):
        tau = max(end_tau,start_tau*gamma**t)
        sum_exp = sum([np.exp(i[1]/(i[0]+1e6)/tau) for i in state])
        res = np.array([np.exp(i[1]/(i[0]+1e6)/tau) / sum_exp for i in state])
        return res
    
    #deterministic policies
    def ucb(self, state, t):
        for i in state:
            if i[0]==0:
                return self.equal_weights(state)
        res = [(i[1]/i[0] + np.sqrt(2*np.log(t+1)/i[0])) for i in state]
        res = np.array(res)
        res_d = np.zeros(len(state))
        res_d[np.argmax(res)] = 1
        return res_d
    def thompson_deterministic(self, state):
        res = [np.random.beta(i[1]+1,i[0]-i[1]+1) for i in state]
        res = np.array(res)
        res_d = np.zeros(len(state))
        res_d[np.argmax(res)] = 1
        return res_d
    
    #thompsons
    def thompson_one(self,state):
        res = [np.random.beta(i[1]+1,i[0]-i[1]+1) for i in state]
        res = np.array(res)
        return res
    def thompson_stochastic(self,state,n=1000):
        l = []
        for i in range(n): l.append(self.thompson_one(state)[None,:])
        l = np.concatenate(l,0)
        is_max = l.max(1)[:,None] == l
        return is_max.mean(0)

In [29]:
env = MusketeerEnv(true_ps = [0.12,0.13,0.14], avg_impressions=400)
a = BanditAgent()
for i in range(20):
    p = a.equal_weights(env.get_state())
    env.step(p)
    t=i
a.equal_weights(env.get_state()), a.randomize(env.get_state()), a.eps_greedy(env.get_state(),t),\
a.softmax(env.get_state(),t), a.thompson_stochastic(env.get_state()), \
a.ucb(env.get_state(),t), a.thompson_deterministic(env.get_state())

(array([0.33333333, 0.33333333, 0.33333333]),
 array([0.60072498, 0.13653053, 0.26274449]),
 array([0.08261686, 0.83476628, 0.08261686]),
 array([0.33261137, 0.33371931, 0.33366932]),
 array([0.002, 0.678, 0.32 ]),
 array([0., 1., 0.]),
 array([0., 0., 1.]))

In [30]:
envs = [MusketeerEnv(true_ps = [0.12,0.13,0.14], avg_impressions=400) for i in range(7)]
a = BanditAgent()
for t in range(200):
    states = [env.get_state() for env in envs]
    actions = [a.equal_weights(states[0]), a.randomize(states[1]),
               a.eps_greedy(states[2],t), a.softmax(states[3],t),
               a.thompson_stochastic(states[4]),
               a.ucb(states[5],t), a.thompson_deterministic(states[6])]
    for i in range(7): envs[i].step(actions[i])
dfs = [env.show_df() for env in envs]
policies = ['equal_weights','randomize','eps_greedy','softmax','thompson_stochastic',
            'ucb','thompson_deterministic']
for i in range(7): dfs[i]['policy'] = policies[i]
df = pd.concat(dfs)[['policy','t','opt_impressions_rate','regret_rate','regret','value_remaining']]

In [31]:
df.tail()

,policy,t,opt_impressions_rate,regret_rate,regret,value_remaining
195,thompson_deterministic,195,0.853583,0.001435,113.110267,0.0
196,thompson_deterministic,196,0.854335,0.001422,112.648052,0.0
197,thompson_deterministic,197,0.854806,0.001420,112.851912,0.0
198,thompson_deterministic,198,0.855497,0.001407,112.355267,0.0
199,thompson_deterministic,199,0.856088,0.001408,112.913086,0.0


In [32]:
df_m = df.melt(id_vars=['policy','t'])
df_m.tail()

,policy,t,variable,value
5595,thompson_deterministic,195,value_remaining,0.0
5596,thompson_deterministic,196,value_remaining,0.0
5597,thompson_deterministic,197,value_remaining,0.0
5598,thompson_deterministic,198,value_remaining,0.0
5599,thompson_deterministic,199,value_remaining,0.0


In [33]:
group='policy')) +
    geom_line() + theme_minimal() + facet_wrap('~variable',scales='free_y'))
g

SyntaxError: invalid syntax (<ipython-input-33-b53a58788c9a>, line 1)